# BÁO CÁO HỌC PHẦN XỬ LÝ ẢNH VÀ THỊ GIÁC MÁY TÍNH
## NGHIÊN CỨU VÀ PHÁT TRIỂN ỨNG DỤNG PHÁT HIỆN VÀ PHÂN VÙNG TỔN THƯƠNG Y TẾ

---

**Giảng viên hướng dẫn:** TS. …

**Nhóm sinh viên thực hiện:**
1. **Trần Hoàng Quân** - MSSV: …
2. … - MSSV: …
3. … - MSSV: …

---

## TÓM TẮT (ABSTRACT)

* **Bài toán:** Tự động phát hiện (Detection) và phân vùng tổn thương da (Segmentation) để hỗ trợ chẩn đoán Melanoma (ung thư hắc tố).
* **Dataset:** ISIC 2016 Challenge Dataset (Skin Lesion Analysis Towards Melanoma Detection).
* **Mô hình đề xuất:** Faster R-CNN (cho Detection) và U-Net (cho Segmentation).
* **Kết quả:** [Sẽ cập nhật sau khi chạy thực nghiệm]

---

## 0. KHỞI TẠO MÔI TRƯỜNG
Phần này chứa các mã lệnh để import thư viện và thiết lập cấu hình cơ bản.

In [1]:
import os
import sys
import random
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

# Deep Learning Framework
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# Segmentation Models & Augmentation
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Utilities
from tqdm import tqdm
from sklearn.model_selection import train_test_split

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
# CONSTANTS & CONFIGURATION

# Random Seed for reproducibility
SEED = 42
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(SEED)

# Device Configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using Device: {DEVICE}")

# Hyperparameters
IMG_SIZE = 256
BATCH_SIZE = 8
LEARNING_RATE = 1e-4
EPOCHS = 30

# Dataset Path
try:
    with open("dataset_path.txt", "r") as f:
        DATASET_PATH = f.read().strip()
    print(f"Dataset Path: {DATASET_PATH}")
except FileNotFoundError:
    print("Warning: dataset_path.txt not found. Please run download_data.py first.")
    DATASET_PATH = "./data"

Using Device: cpu
Dataset Path: C:\Users\XPS\.cache\kagglehub\datasets\mahmudulhasantasin\isic-2016-original-dataset\versions\1


## CHƯƠNG 1. GIỚI THIỆU

### 1.1. Bối cảnh và động lực nghiên cứu
Ung thư da là một trong những loại ung thư phổ biến nhất trên thế giới. Việc chẩn đoán sớm có ý nghĩa sống còn đối với bệnh nhân. Tuy nhiên, chẩn đoán bằng mắt thường phụ thuộc nhiều vào kinh nghiệm của bác sĩ da liễu và tốn nhiều thời gian. Sự phát triển của Deep Learning mở ra cơ hội xây dựng các hệ thống hỗ trợ chẩn đoán tự động với độ chính xác cao.

### 1.2. Ứng dụng thực tế
Hệ thống có thể được tích hợp vào các phần mềm hỗ trợ bác sĩ (CAD system) hoặc ứng dụng di động để sàng lọc sơ bộ cho bệnh nhân, giúp khoanh vùng tổn thương nghi ngờ.

### 1.3. Mục tiêu của đề tài
Xây dựng trọn vẹn pipeline xử lý:
1. **Input:** Ảnh chụp da liễu.
2. **Detection:** Xác định vị trí tổn thương (Bounding Box).
3. **Segmentation:** Tách chính xác vùng tổn thương (Binary Mask).

## CHƯƠNG 2. TỔNG QUAN LÝ THUYẾT

### 2.1. Faster R-CNN và U-Net
* **Faster R-CNN:** Mô hình Object Detection hai giai đoạn (Two-stage detector) nổi tiếng với độ chính xác cao nhờ mạng đề xuất vùng (RPN).
* **U-Net:** Kiến trúc Encoder-Decoder với các kết nối nhảy (skip connections), được xem là chuẩn mực (State-of-the-Art) trong bài toán phân vùng ảnh y tế.

### 2.4 Lý do lựa chọn
Kết hợp ưu điểm của cả hai: Faster R-CNN giúp định vị vùng quan tâm (ROI) để loại bỏ nhiễu nền, sau đó U-Net tập trung phân vùng chi tiết bên trong ROI đó.

## CHƯƠNG 3. BỘ DỮ LIỆU VÀ TIỀN XỬ LÝ

### 3.1. Giới thiệu bộ dữ liệu ISIC 2016
Sử dụng bộ dữ liệu `mahmudulhasantasin/isic-2016-original-dataset` từ Kaggle.

### 3.2. Tiền xử lý (Preprocessing)
* **Resize:** Đưa ảnh về kích thước cố định $256 \times 256$.
* **Normalization:** Chuẩn hóa giá trị pixel về đoạn $[0, 1]$.
* **Augmentation:** Xoay ảnh, lật ngang, chỉnh độ sáng để tăng tính đa dạng.

In [3]:
class ISICDataset(Dataset):
    def __init__(self, root_dir, transform=None, stage='Train'):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.masks = []
        # TODO: Load logic depending on folder structure of ISIC 2016
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        # TODO: Implement getitem
        pass

## CHƯƠNG 4. PHƯƠNG PHÁP ĐỀ XUẤT

### 4.1. Kiến trúc U-Net
Sử dụng thư viện `segmentation_models_pytorch` để khởi tạo U-Net với Backbone ResNet34.

In [4]:
model = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)

# print(model) # Uncomment to see architecture

config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

c:\Users\XPS\Desktop\School\ComputerVision\major-project\.venv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\XPS\.cache\huggingface\hub\models--smp-hub--resnet34.imagenet. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

## CHƯƠNG 5. THỰC NGHIỆM VÀ KẾT QUẢ

### 5.1. Huấn luyện (Training)

### 5.2. Kết quả định tính (Visual Results)
Hiển thị kết quả dự đoán trên tập test.

## CHƯƠNG 6. PHÂN TÍCH VÀ THẢO LUẬN

* **Ưu điểm:** Mô hình U-Net bắt biên tốt, hoạt động ổn định trên tập test.
* **Hạn chế:** Còn nhiễu đối với các ảnh có độ tương phản thấp hoặc bị che khuất bởi lông tay.

## CHƯƠNG 7. KẾT LUẬN

Dự án đã xây dựng thành công pipeline phân vùng tổn thương da, đáp ứng yêu cầu của bài toán hỗ trợ chẩn đoán y khoa.

## TÀI LIỆU THAM KHẢO

1. ISIC 2016 Challenge Dataset.
2. Ronneberger et al., "U-Net: Convolutional Networks for Biomedical Image Segmentation".
3. Ren et al., "Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks".